# Workshop: Multi-Agent система поиска лучшего курса валют

## Добро пожаловать на 30-минутный workshop по созданию мульти-агентных систем!

### Что мы будем изучать:
- Фреймворк **smolagents** от Hugging Face
- Создание кастомных инструментов для агентов
- Интеграцию с внешними API (Serper.dev, ЦБ РФ)
- Координацию работы между несколькими специализированными агентами

### Сценарий:
Создадим систему из 4 агентов, которые помогут найти лучший курс обмена USD на рубли:
1. **CurrencyAgent** - получает официальный курс ЦБ РФ
2. **LocationAgent** - находит банки рядом с пользователем
3. **BankRatesAgent** - собирает курсы обмена в банках
4. **DecisionAgent** - анализирует и рекомендует лучший вариант

## Шаг 1: Установка и импорты (5 минут)

Убедитесь, что у вас установлены все необходимые зависимости:
```bash
pip install -r requirements.txt
```

In [ ]:
# Импорты
import os
from dotenv import load_dotenv
from smolagents import CodeAgent, VisitWebpageTool
from cloud_ru_model import CloudRuModel, test_cloud_ru_connection
import warnings
warnings.filterwarnings('ignore')

# Загружаем переменные окружения
load_dotenv()

# Импортируем наши кастомные инструменты
from custom_tools import (
    SerperSearchTool, 
    CBRCurrencyTool, 
    UserInputTool, 
    AddressToCoordsTool,
)

print("✅ Все модули импортированы успешно!")

✅ Все модули импортированы успешно!


## Шаг 2: Настройка API ключей

**Важно:** Создайте файл `.env` на основе `.env.example` и заполните ваши API ключи:
- **Cloud.ru Foundation Models API** - основной провайдер LLM (российский)
- Serper.dev API ключ для поиска в Google
- OpenAI API ключ (опционально, как fallback)

In [ ]:
# Проверяем наличие API ключей
def check_api_keys():
    # Проверяем Cloud.ru API (основной)
    cloud_ru_api = os.getenv("CLOUD_RU_API_KEY")
    
    # Проверяем дополнительные API
    other_keys = {
        "SERPER_API_KEY": os.getenv("SERPER_API_KEY")
    }
    
    print("🏢 Cloud.ru Foundation Models API (основной провайдер):")
    if cloud_ru_api:
        print(f"  ✅ API ключ настроен (начинается с {cloud_ru_api[:10]}...)")
        cloud_ru_ok = True
    else:
        print(f"  ❌ API ключ НЕ НАСТРОЕН! Установите CLOUD_RU_API_KEY")
        cloud_ru_ok = False
    
    print("\n🔧 Дополнительные API:")
    for key_name, key_value in other_keys.items():
        if key_value:
            print(f"  ✅ {key_name}: настроен")
        else:
            status = "НЕ НАСТРОЕН" if key_name == "SERPER_API_KEY" else "НЕ НАСТРОЕН (опционально)"
            print(f"  ❌ {key_name}: {status}")
    
    serper_ok = bool(other_keys["SERPER_API_KEY"])
    
    return cloud_ru_ok and serper_ok, cloud_ru_ok, serper_ok

all_keys_ok, cloud_ru_ok, serper_ok = check_api_keys()
print(f"\nГотовность к работе: {'✅' if all_keys_ok else '⚠️ частично' if cloud_ru_ok else '❌'}")

# Тестируем подключение к Cloud.ru
if cloud_ru_ok:
    print("\n🧪 Тестирование подключения к Cloud.ru API...")
    cloud_connection_ok = test_cloud_ru_connection()
else:
    cloud_connection_ok = False
    print("\n❌ Невозможно протестировать Cloud.ru API - отсутствует ключ")

🏢 Cloud.ru Foundation Models API (основной провайдер):
  ✅ API ключ настроен (начинается с YWYyNGYzZD...)

🔧 Дополнительные API:
  ✅ SERPER_API_KEY: настроен

Готовность к работе: ✅

🧪 Тестирование подключения к Cloud.ru API...
🧪 Тестируем подключение к Cloud.ru API...
✅ Подключение успешно! Ответ: Да
📊 Модель: zai-org/GLM-4.5
🔗 URL: https://foundation-models.api.cloud.ru/v1


## Шаг 3: Создание специализированных агентов (15 минут)

### 3.1 CurrencyAgent - Агент для получения официального курса валют

In [8]:
# Создаем модель для наших агентов
# Используем Cloud.ru с моделью GLM-4.5 (zai-org)

if cloud_ru_ok:
    print("🏢 Используем Cloud.ru Foundation Models API")
    print("🤖 Модель: zai-org/GLM-4.5 (355B параметров, оптимизирована для агентов)")
    try:
        model = CloudRuModel(
            model_name="zai-org/GLM-4.5",  # Используем GLM-4.5 вместо GPT
            temperature=0.5,
            max_tokens=5000,
            top_p=0.95,
            presence_penalty=0
        )
        print("✅ Модель GLM-4.5 инициализирована (без fallback)")
    except Exception as e:
        print(f"❌ Ошибка инициализации GLM-4.5: {e}")
        raise
else:
    print("❌ Cloud.ru API недоступен")
    raise ValueError("Требуется настройка Cloud.ru API в файле .env")

print(f"📊 Информация о модели: {getattr(model, 'get_model_info', lambda: {'provider': 'Unknown'})()}")

# Агент для получения официального курса валют
currency_agent = CodeAgent(
    tools=[CBRCurrencyTool()],
    model=model
)

print("✅ CurrencyAgent создан с GLM-4.5!")

# Тестируем агента
print("\n🧪 Тестирование CurrencyAgent с GLM-4.5:")
result = currency_agent.run("Ты - CurrencyAgent, специалист по валютным курсам. Получи актуальный курс доллара США к рублю от ЦБ РФ и предоставь точную информацию о курсе и дате его установления. Отвечай на русском языке.")
print(result)

🏢 Используем Cloud.ru Foundation Models API
🤖 Модель: zai-org/GLM-4.5 (355B параметров, оптимизирована для агентов)
✅ Модель GLM-4.5 инициализирована (без fallback)
📊 Информация о модели: {'provider': 'Cloud.ru', 'model_name': 'zai-org/GLM-4.5', 'base_url': 'https://foundation-models.api.cloud.ru/v1', 'temperature': 0.5, 'max_tokens': 5000, 'top_p': 0.95}
✅ CurrencyAgent создан с GLM-4.5!

🧪 Тестирование CurrencyAgent с GLM-4.5:


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Ты - CurrencyAgent, специалист по валютным курсам. Получи актуальный курс доллара США к рублю от ЦБ РФ и        │
│ предоставь точную информацию о курсе и дате его установления. Отвечай на русском языке.                         │
│                                                                                                                 │
╰─ CloudRuModel - None ───────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  currency_rates = cbr_currency()                                                                                  
  print(currency_rates)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Официальный курс ЦБ РФ на 2025-08-26:
USD/RUB: 80.6842 руб.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Актуальный курс доллара США к рублю по данным ЦБ РФ на 26 августа 2025 года составляет 80.6842     
  рубля за 1 доллар США.")                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Актуальный курс доллара США к рублю по данным ЦБ РФ на 26 августа 2025 года составляет 80.6842 
рубля за 1 доллар США.

[Step 2: Duration 4.21 seconds]

Актуальный курс доллара США к рублю по данным ЦБ РФ на 26 августа 2025 года составляет 80.6842 рубля за 1 доллар США.


### 3.2 LocationAgent - Агент для поиска банков по местоположению

In [ ]:
# Агент для работы с местоположением и поиском банков
location_agent = CodeAgent(
    tools=[SerperSearchTool(), AddressToCoordsTool()],
    model=model,
    name="location_agent",
    description="Отвечает за поиск банков рядом с адресом пользователя и получения их географических координат",
)

print("✅ LocationAgent создан!")

✅ LocationAgent создан!


### 3.3 BankRatesAgent - Агент для получения курсов в банках

In [ ]:
# Агент для получения курсов валют в банках
bank_rates_agent = CodeAgent(
    tools=[VisitWebpageTool()], 
    model=model,
    name="bank_rates_agent",
    description="Отвечает за поиск курса валют в банках",
)

banki_ros_url = 'https://myfin.by/currency/moskva'

print("✅ BankRatesAgent создан!")

✅ BankRatesAgent создан!


In [21]:
# Тестируем агента
print("\n🧪 Тестирование BankRatesAgent:")
result = bank_rates_agent.run(f"""Ты - BankRatesAgent, специалист по банковским курсам валют. 
Получи курсы покупки и продажи USD в различных банках, используй {banki_ros_url}. Парси текст с учётом правильной кодировки сайта. Отвечай на русском языке.""")
print(result)


🧪 Тестирование BankRatesAgent:


╭────────────────────────────────────────── New run - bank_rates_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ Ты - BankRatesAgent, специалист по банковским курсам валют.                                                     │
│ Получи курсы покупки и продажи USD в различных банках, используй https://myfin.by/currency/moskva. Парси текст  │
│ с учётом правильной кодировки сайта. Отвечай на русском языке.                                                  │
│                                                                                                                 │
╰─ CloudRuModel - None ───────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  webpage_content = visit_webpage("https://myfin.by/currency/moskva")                                              
  print(webpage_content)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Курсы валют в банках Москвы на сегодня, выгодный курс обмена валют в обменниках Москвы, доллар и евро 
![pixel](https://www.facebook.com/tr?id=1307149983547936&ev=PageView&noscript=1)

![cookies](/files_content/img/components/cookie/cookie.svg)

Согласие на обработку cookie

Сайт использует файлы cookie для обеспечения удобства пользователей сайта, его улучшения, предоставления 
персонализированных рекомендаций. Вы можете настроить обработку cookie. Отозвать соглаcие можно на странице 
[Политики обработки файлов cookie](/site/cookies-policy).

СогласенОтклонить

Сайт может перестать корректно работать

Часть функционала сайта перестанет работать. Мы не сможем хранить персональные настройки и рекомендации.

Для полноценной работы всего сайта рекомендуем принять все cookie или выполнить настройку.

Отклонить

Настроить

Принять соглашение

[![MYFIN](/images/new/logo-myfin--new.svg "MYFIN")](/ "На главную Myfin.by")

* [Деньги на Белджи](https://myfin.by/kredity/belgee?utm_source=myfin&utm_medium=organic&utm_campaign=header)
* [Все банки](/banki)
* [Реклама](https://ads.myfin.by/?utm_source=myfin&utm_medium=organic&utm_campaign=preheader)
* [Стать автором](/site/contacts?utm_source=myfin&utm_medium=organic&utm_campaign=preheader)
* Войти

[![MYFIN](/images/new/logo-short.svg "MYFIN")](/ "На главную Myfin.by")

* [Курсы валют](/currency/minsk?utm_source=myfin&utm_medium=organic&utm_campaign=menu)

  [Курсы валют](/currency/minsk?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Забронировать курс](/zabronirovat-kurs?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Конвертер](/converter?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курсы
    НБРБ](/bank/kursy_valjut_nbrb?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курсы ЦБРФ](/currency/cb-rf?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Торги
    БВФБ](/currency/torgi-na-bvfb?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курсы для
    больших сумм](/currency/large-amounts?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курсы валют в
    Москве](/currency/moskva?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + gag
  + [Курс доллара](/currency/usd?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курс евро](/currency/eur?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курс рубля](/currency/rub?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Доллар/Рубль](/currency/usdrub?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Евро/Доллар](/currency/eurusd?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курс злотого](/currency/pln?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курс гривны](/currency/uah?utm_source=myfin&utm_medium=organic&utm_campaign=menu)

  + [Минск](/currency/minsk?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Брест](/currency/brest?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Витебск](/currency/vitebsk?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Гомель](/currency/gomel?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Гродно](/currency/grodno?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Могилёв](/currency/mogilev?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Пинск](/currency/pinsk?utm_source=myfin&utm_medium=organic&utm_campaign=menu)

  [Курсы криптовалют](/crypto-rates?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курс
    Bitcoin](/crypto-rates/bitcoin?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Курс
    Ethereum](/crypto-rates/ethereum?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
* [Новости](/article?utm_source=myfin&utm_medium=organic&utm_campaign=menu)

  Новости
  + [Кошелек](/article/money?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Рынки](/article/rynki?utm_source=myfin&utm_medium=organic&utm_campaign=menu)
  + [Банки](/article/banki?utm_so

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Формируем итоговый ответ                                                                                       
  result = """                                                                                                     
  # Курсы покупки и продажи USD в банках Москвы                                                                    
                                                                                                                   
  По данным с сайта myfin.by, на сегодня в банках Москвы доступны следующие курсы USD:                             
                                                                                                                   
  ## Лучшие курсы USD:                                                                                             
                                                                                                                   
  * **Лучший курс покупки USD**: 80.75 руб. в банке "Кетовский"                                                    
  * **Лучший курс продажи USD**: 79.85 руб. в банке "Ак Барс Банк"                                                 
                                                                                                                   
  ## Средние показатели по рынку:                                                                                  
                                                                                                                   
  * **Средний курс покупки USD**: 78.72 руб.                                                                       
  * **Средний курс продажи USD**: 82.71 руб.                                                                       
  * **Средний спред** (разница между покупкой и продажей): 3.99 руб.                                               
                                                                                                                   
  ## Топ-5 банков с лучшими курсами покупки USD:                                                                   
                                                                                                                   
  1. Кетовский - 80.75 руб.                                                                                        
  2. Дружба - 80.70 руб.                                                                                           
  3. ЦентроКредит - 80.50 руб.                                                                                     
  4. Внешфинбанк - 80.30 руб.                                                                                      
  5. Авангард, БыстроБанк - 80.20 руб.                                                                             
                                                                                                                   
  ## Топ-5 банков с лучшими курсами продажи USD:                                                                   
                                                                                                                   
  1. Ак Барс Банк - 79.85 руб.                                                                                     
  2. КАМКОМБАНК - 79.99 руб.                                                                                       
  3. Трансстройбанк - 80.30 руб.                                                                                   
  4. Агророс - 80.64 руб.                                                                                          
  5. Солид Банк - 80.85 руб.                                                                                       
                                                                                                                   
  Информация о курсах валют носит справочный характер и м

Out - Final answer: 
# Курсы покупки и продажи USD в банках Москвы

По данным с сайта myfin.by, на сегодня в банках Москвы доступны следующие курсы USD:

## Лучшие курсы USD:

* **Лучший курс покупки USD**: 80.75 руб. в банке "Кетовский"
* **Лучший курс продажи USD**: 79.85 руб. в банке "Ак Барс Банк"

## Средние показатели по рынку:

* **Средний курс покупки USD**: 78.72 руб.
* **Средний курс продажи USD**: 82.71 руб.
* **Средний спред** (разница между покупкой и продажей): 3.99 руб.

## Топ-5 банков с лучшими курсами покупки USD:

1. Кетовский - 80.75 руб.
2. Дружба - 80.70 руб.
3. ЦентроКредит - 80.50 руб.
4. Внешфинбанк - 80.30 руб.
5. Авангард, БыстроБанк - 80.20 руб.

## Топ-5 банков с лучшими курсами продажи USD:

1. Ак Барс Банк - 79.85 руб.
2. КАМКОМБАНК - 79.99 руб.
3. Трансстройбанк - 80.30 руб.
4. Агророс - 80.64 руб.
5. Солид Банк - 80.85 руб.

Информация о курсах валют носит справочный характер и может быть изменена банком в течение дня. Актуальность курса 
и информацию о наличии валюты уточняйте по телефонам банка.

[Step 4: Duration 5.79 seconds]


# Курсы покупки и продажи USD в банках Москвы

По данным с сайта myfin.by, на сегодня в банках Москвы доступны следующие курсы USD:

## Лучшие курсы USD:

* **Лучший курс покупки USD**: 80.75 руб. в банке "Кетовский"
* **Лучший курс продажи USD**: 79.85 руб. в банке "Ак Барс Банк"

## Средние показатели по рынку:

* **Средний курс покупки USD**: 78.72 руб.
* **Средний курс продажи USD**: 82.71 руб.
* **Средний спред** (разница между покупкой и продажей): 3.99 руб.

## Топ-5 банков с лучшими курсами покупки USD:

1. Кетовский - 80.75 руб.
2. Дружба - 80.70 руб.
3. ЦентроКредит - 80.50 руб.
4. Внешфинбанк - 80.30 руб.
5. Авангард, БыстроБанк - 80.20 руб.

## Топ-5 банков с лучшими курсами продажи USD:

1. Ак Барс Банк - 79.85 руб.
2. КАМКОМБАНК - 79.99 руб.
3. Трансстройбанк - 80.30 руб.
4. Агророс - 80.64 руб.
5. Солид Банк - 80.85 руб.

Информация о курсах валют носит справочный характер и может быть изменена банком в течение дня. Актуальность курса и информацию о наличии валюты уточн

### 3.4 DecisionAgent - Агент для анализа и принятия решений

In [ ]:
# Агент для анализа и принятия решений

manager_agent = CodeAgent(
    tools=[UserInputTool()],  # Этот агент работает только с входом пользователя
    model=model,
    managed_agents=[location_agent, bank_rates_agent],
)

result = manager_agent.run(f"""Ты - ManagerAgent. Помощник выбора банка рядом с пользователем для обмена курса валют.
Предоставляй четкие рекомендации: лучший банк для покупки валюты (наименьший курс продажи), 
лучший банк для продажи валюты (наибольший курс покупки), общие выводы и советы. 
Отвечай на русском языке. Руководствуйся следующей логикой что если купить 100$ то лучше сделать это поближе к дому, 
а если 5000$ нужно купить то нужен выгодный курс и можно подальше от дома до 10 километров расстояния.
Используй {banki_ros_url}, для поиска курса валют. Парси текст с учётом правильной кодировки сайта.""")

print("✅ DecisionAgent создан!")
print("\n🎉 Все агенты готовы к работе!")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Ты - ManagerAgent. Помощник выбора банка рядом с пользователем для обмена курса валют.                          │
│ Предоставляй четкие рекомендации: лучший банк для покупки валюты (наименьший курс продажи),                     │
│ лучший банк для продажи валюты (наибольший курс покупки), общие выводы и советы.                                │
│ Отвечай на русском языке. Руководствуйся следующей логикой что если купить 100$ то лучше сделать это поближе к  │
│ дому,                                                                                                           │
│ а если 5000$ нужно купить то нужен выгодный курс и можно подальше от дома до 10 километров расстояния.          │
│ Используй https://myfin.by/currency/moskva, https://bankiros.ru/currency/usd для поиска курса валют             │
│                                                                                                                 │
╰─ CloudRuModel - None ───────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  user_location = user_input("Пожалуйста, укажите ваш адрес или местоположение для поиска ближайших банков:")      
  print(f"Местоположение пользователя: {user_location}")                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Местоположение пользователя: Первомайская 109/2

Out: None

[Step 1: Duration 10.99 seconds]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  recommendations = """                                                                                            
  # Рекомендации по выбору банка для обмена валюты                                                                 
                                                                                                                   
  ## Лучшие банки для обмена валюты рядом с адресом Первомайская 109/2:                                            
                                                                                                                   
  ### Для покупки долларов США (USD):                                                                              
  **СберБанк** - наилучший вариант                                                                                 
  - Курс продажи: 81.15 руб. за 1 USD                                                                              
  - Расстояние: 3.17 км                                                                                            
  - Адрес: Москва, Щёлковское шоссе, 75, ТЦ «Щёлковский»                                                           
  - Режим работы: пн-сб: 10:00-21:00; вс: 10:30-19:00                                                              
                                                                                                                   
  ### Для продажи долларов США (USD):                                                                              
  **Банк Русский Стандарт** - наилучший вариант                                                                    
  - Курс покупки: 80.0 руб. за 1 USD                                                                               
  - Расстояние: 3.17 км                                                                                            
  - Адрес: Москва, Щёлковское шоссе, 75, ТЦ «Щёлковский»                                                           
  - Режим работы: пн-сб с 10:00 до 21:00; вс с 11:30 до 19:30                                                      
                                                                                                                   
  ## Общие выводы и советы:                                                                                        
                                                                                                                   
  ### В зависимости от суммы обмена:                                                                               
  1. **Для небольших сумм (до 100$):**                                                                             
     - Рекомендую обратиться в **Альфа-Банк** (ближайший, всего 1.43 км)                                           
     - Адрес: Москва, 9-я Парковая улица, 23                                                                       
     - Режим работы: пн-пт 10:00–20:00; сб 10:00–18:00                                                             
     - Примечание: уточните курсы непосредственно в банке, так как они не были найдены в открытых источниках       
                                                                                                                   
  2. **Для крупных сумм (от 5000$):**                                                                              
     - Для покупки USD: **СберБанк** (самый выгодный курс продажи)                                                 
     - Для продажи USD: **Банк Русский Стандарт** (самый выгодный курс покупки)                                    
     - Расстояние до этих банков (3.17 км) оправдано экономией на крупной сумме                                    
                                                                                                                   
  ### Дополнительные советы:                             

Out - Final answer: 
# Рекомендации по выбору банка для обмена валюты

## Лучшие банки для обмена валюты рядом с адресом Первомайская 109/2:

### Для покупки долларов США (USD):
**СберБанк** - наилучший вариант
- Курс продажи: 81.15 руб. за 1 USD
- Расстояние: 3.17 км
- Адрес: Москва, Щёлковское шоссе, 75, ТЦ «Щёлковский»
- Режим работы: пн-сб: 10:00-21:00; вс: 10:30-19:00

### Для продажи долларов США (USD):
**Банк Русский Стандарт** - наилучший вариант
- Курс покупки: 80.0 руб. за 1 USD
- Расстояние: 3.17 км
- Адрес: Москва, Щёлковское шоссе, 75, ТЦ «Щёлковский»
- Режим работы: пн-сб с 10:00 до 21:00; вс с 11:30 до 19:30

## Общие выводы и советы:

### В зависимости от суммы обмена:
1. **Для небольших сумм (до 100$):**
   - Рекомендую обратиться в **Альфа-Банк** (ближайший, всего 1.43 км)
   - Адрес: Москва, 9-я Парковая улица, 23
   - Режим работы: пн-пт 10:00–20:00; сб 10:00–18:00
   - Примечание: уточните курсы непосредственно в банке, так как они не были найдены в открытых источниках

2. **Для крупных сумм (от 5000$):**
   - Для покупки USD: **СберБанк** (самый выгодный курс продажи)
   - Для продажи USD: **Банк Русский Стандарт** (самый выгодный курс покупки)
   - Расстояние до этих банков (3.17 км) оправдано экономией на крупной сумме

### Дополнительные советы:
- Курсы валют могут меняться в течение дня, поэтому перед совершением операции рекомендуется уточнить актуальные 
курсы непосредственно в банках
- Некоторые банки предлагают специальные условия для крупных сумм обмена
- Перед обменом валюты рекомендуется уточнить наличие купюр в отделении банка
- Обратите внимание, что СберБанк и Банк Русский Стандарт находятся в одном торговом центре (ТЦ «Щёлковский»), что 
удобно для сравнения условий

### Сравнение всех найденных банков:
1. **ВТБ Банк** (2.58 км)
   - Покупка USD: 79.4 руб.
   - Продажа USD: 82.4 руб.

2. **СберБанк** (3.17 км)
   - Покупка USD: 76.6 руб.
   - Продажа USD: 81.15 руб.

3. **Банк Русский Стандарт** (3.17 км)
   - Покупка USD: 80.0 руб.
   - Продажа USD: 84.0 руб.

4. **Альфа-Банк** (1.43 км)
   - Курсы не найдены, рекомендуется уточнить непосредственно в банке

[Step 5: Duration 16.82 seconds]

✅ DecisionAgent создан!

🎉 Все агенты готовы к работе!


## Заключение и дальнейшее развитие

### Что мы изучили:
- ✅ Фреймворк **smolagents** от Hugging Face
- ✅ Создание кастомных инструментов (Tools) для агентов
- ✅ Интеграцию с внешними API (Serper.dev, ЦБ РФ)
- ✅ Координацию работы между специализированными агентами
- ✅ Построение end-to-end решения с мульти-агентной архитектурой

### Возможные улучшения:
1. **Добавить больше источников данных** - другие поисковые API, банковские API
2. **Улучшить парсинг результатов** - более точное извлечение информации о банках
3. **Добавить геолокацию** - автоматическое определение местоположения пользователя
4. **Кэширование данных** - сохранение результатов для повторного использования
5. **Визуализация** - графики курсов, карты с банками
6. **Мониторинг курсов** - отслеживание изменений в реальном времени

### Дополнительные ресурсы:
- [Документация smolagents](https://huggingface.co/docs/smolagents/index)
- [Serper.dev API Documentation](https://serper.dev/)
- [ЦБ РФ API](https://www.cbr.ru/development/dws/)

---

**Спасибо за участие в workshop! 🎉**

Теперь вы знаете основы создания мульти-агентных систем с smolagents!

## Бонус: Экспериментируйте дальше!

Попробуйте модифицировать агентов или добавить новые возможности:

In [ ]:
# Пример: создайте своего собственного агента
# experimental_agent = CodeAgent(
#     tools=[...],
#     model=model,
#     system_prompt="Ваш промпт..."
# )

print("💡 Место для ваших экспериментов!")
print("Попробуйте:")
print("- Изменить промпты агентов")
print("- Добавить новые инструменты")
print("- Создать агента для другой валюты")
print("- Добавить сравнение с курсами онлайн-обменников")